![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

Short project description. Your bottom line up front (BLUF) insights.

## Business Understanding

The customer of this project is FutureProduct Advisors, a consultancy that helps their customers develop innovative and new consumer products. FutureProduct’s customers are increasingly seeking help from their consultants in go-to-market activities. 

FutureProduct’s consultants can support these go-to-market activities, but the business does not have all the infrastructure needed to support it. Their biggest ask is for a tool to help them find interesting, up-and-coming music to accompany social posts and online ads for go-to-market promotions. 

**Stakeholders**

- FutureProduct Managing Director: oversees their consulting practice and is sponsoring this project.
- FutureProduct Senior Consultants: the actual users of the prospective tool. A small subset of the consultants will pilot the prototype tool.
- My consulting leadership: sponsors of this effort; will provide oversight and technical input of the project as needed.

**Primary Goals**

1.	Build a data tool that can evaluate any song in the Billboard Hot 100 list and make predictions about:
    -	The song’s position on the Hot 100 list 4 weeks in the future
    -	The song’s highest position on the list in the next 6 months
2.	Create a rubric that lists the 3 most important factors for songs’ placement on the Hot 100 list for each hear from 2000 to 2021.


## Data Understanding

Billboard Hot 100 weekly charts (Kaggle): https://www.kaggle.com/datasets/thedevastator/billboard-hot-100-audio-features

I’ve chosen this dataset because it has a direct measurement of song popularity (the Hot 100 list) and because its long history gives significant context to a song’s positioning in a given week.
The features list gives a wide range of song attributes to explore and enables me to determine what features most significantly contribute to a song’s popularity and how that changes over time.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import ast
from collections import Counter

import xgboost as xgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(42)

In [ ]:
df_hotlist_all = pd.read_csv('Data/Hot Stuff.csv')
df_features_all = pd.read_csv('Data/Hot 100 Audio Features.csv')

In [ ]:
# exploring hotlist df
df_hotlist_all.info()

In [ ]:
# exploring features df
df_features_all.info()

#### Exploratory Data Analysis

In [ ]:
df_cleaned['Max_Peak_Position'].tail(20)

In [ ]:
peak_pos_dist = df_cleaned['Max_Peak_Position'].value_counts()
peak_pos_dist

In [ ]:
plt.hist(peak_pos_dist)
plt.xlabel('')

## Data Preparation

### Initial Data Selection and Feature Engineering

In [ ]:
# removing hotlist df attributes that will not be used in cleaning or analysis
df_hotlist_all = df_hotlist_all.drop(['index', 'url', 'Song', 'Performer'], axis=1)
# converting WeekID to datetime
df_hotlist_all['WeekID'] = pd.to_datetime(df_hotlist_all['WeekID'], errors='coerce')
df_hotlist_all = df_hotlist_all.sort_values(by='WeekID')

# creating a new hotlist df with only complete year data from 2000 - 2020, the time period being studied
df_hotlist_2000s = df_hotlist_all.loc[(df_hotlist_all['WeekID'] > '1999-12-31') & (df_hotlist_all['WeekID'] < '2021-01-01')]

# adding a column to calculate the week over week change in rank
def diff(a, b):
    return a - b

df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s.apply(lambda x: diff(x['Week Position'], x['Previous Week Position']), axis=1)
# replacing NaNs with 0
df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s['Rank_Change'].fillna(0)

# removing features df attributes that will not be used in cleaning or analysis
df_features_all = df_features_all.drop(['index', 'Performer', 'Song', 'spotify_track_album', 
                                        'spotify_track_preview_url', 'spotify_track_explicit', 
                                        'spotify_track_popularity'], axis=1)

In [ ]:
# new df with the max weekly rank change for each song in df_hotlist_2000s
df_max_rank_change = df_hotlist_2000s.groupby('SongID', as_index=False)['Rank_Change'].max()
df_max_rank_change.rename(columns={'Rank_Change': 'Max_Rank_Change'}, inplace=True)
df_max_rank_change.set_index('SongID', inplace=True)

# new df with the max peak rank for each song in df_hotlist_2000s
df_max_peak_pos = df_hotlist_2000s.groupby('SongID', as_index=False)['Peak Position'].max()
df_max_peak_pos.rename(columns={'Peak Position': 'Max_Peak_Position'}, inplace=True)
df_max_peak_pos.set_index('SongID', inplace=True)

# ensuring these new dfs have no null values
df_max_rank_change['Max_Rank_Change'].isna().sum(), df_max_peak_pos['Max_Peak_Position'].isna().sum()

In [ ]:
# extracting full list of songs in the time period being studied
songid_list = df_hotlist_2000s['SongID'].unique()

# creating a features df with only songs in df_hotlist_2000s
df_features_2000s = df_features_all[df_features_all['SongID'].isin(songid_list)]

# checking for duplicates
print(len(df_features_2000s))
print(len(pd.unique(df_features_2000s['SongID'])))

In [ ]:
# removing duplicates and rechecking
df_features_2000s = df_features_2000s.drop_duplicates(subset='SongID')

print(len(df_features_2000s))
print(len(pd.unique(df_features_2000s['SongID'])))

In [ ]:
# adding max peak position to features df
df_2000s_data = df_features_2000s.join(df_max_peak_pos, on='SongID')

# adding max rank change to features df
df_2000s_data = df_2000s_data.join(df_max_rank_change, on='SongID')

# removing entries with missing values and defining as a new df
df_cleaned = df_2000s_data[df_2000s_data.notna().all(axis=1)]
df_cleaned.info()

### Feature Engineering for Genre

The dataset has genre in a single column; the entry for each song has a variety of genres listed in that single column. In order to explore genre, I'll need to break this field out.

In [ ]:
# generating a df with unique genre names
unique_genres = list(set(
    genre 
    for genre_string in df_cleaned['spotify_genre'] 
    if pd.notna(genre_string)
    for genre in ast.literal_eval(genre_string)
))

df_unique_genres = pd.DataFrame(unique_genres, columns=['genre'])

# adding counts of each unique genre name
# Extract all genres (with duplicates) and count them
all_genres_list = []
for genre_string in df_cleaned['spotify_genre']:
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        all_genres_list.extend(genre_list)

# Count occurrences
genre_counts = Counter(all_genres_list)

# Map counts to genres dataframe
df_unique_genres['count'] = df_unique_genres['genre'].map(genre_counts)
df_unique_genres = df_unique_genres.sort_values('count', ascending=False)

In [ ]:
# writing to csv for easier review of the data
df_unique_genres.to_csv('genre_counts.csv', index=False)

After reviewing the full set of genre counts, I created a new csv that contains genres which appear in 50 or more song entries.

In [ ]:
# loading list of genres with 50 or more instances in df_cleaned
df_genres_50_up = pd.read_csv('genre_counts_50+inst.csv')

# converting df to list
final_genres_list = df_genres_50_up['genre'].tolist()

# manually one-hot encoding each genre

# creating each new genre column and initializing to 0
for genre in final_genres_list:
    df_cleaned[genre] = 0

# iterating through rows to set values to 1 when genre column appears in original spotify_genre column
for idx, genre_string in enumerate(df_cleaned['spotify_genre']):
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        for genre in genre_list:
            df_cleaned.at[idx, genre] = 1

In [ ]:
# reviewing full df 
pd.set_option('display.max_columns', None)
df_cleaned.head(3)

I created two datasets: one containing genre and one without. This will allow me to model this data with and without genre.

In [ ]:
# removing fields used for prep/cleaning but not needed for analysis
df_cleaned = df_cleaned.drop(['SongID', 'spotify_genre', 'spotify_track_id'], axis=1)

# my code added columns for all genres in spotify_genre, removing unwanted columns and creating a clean df with genre
last_col_to_keep_genre = 'emo rap'
df_cleaned_genre = df_cleaned.loc[:, :last_col_to_keep_genre]
# removing NaN rows
df_cleaned_genre = df_cleaned_genre.dropna()

# creating a clean df for analysis without genre
last_col_to_keep_no_genre = 'Max_Rank_Change'
df_cleaned_no_genre = df_cleaned.loc[:, :last_col_to_keep_no_genre]
# removing NaN rows
df_cleaned_genre = df_cleaned_genre.dropna()
df_cleaned_no_genre = df_cleaned_no_genre.dropna()

df_cleaned_genre.info(), df_cleaned_no_genre.info()

### Features and Target Variables

I'm prepping 4 versions for XGBoost and k-NN:

1. Max Peak Position, no genre (1__1 variables)
2. Max Peak Position, with genre (1_2 variables)
3. Max Rank Change, no genre (2_1 variables)
4. Max Rank Change, with genre (2_2 variables)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, no genre
X1_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y1_1 = df_cleaned_no_genre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X1_1_train, X1_1_test, y1_1_train, y1_1_test = train_test_split(X1_1, y1_1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X1_1_train_scaled = scaler.fit_transform(X1_1_train)
X1_1_test_scaled = scaler.fit_transform(X1_1_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, including genre
X1_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y1_2 = df_cleaned_genre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X1_2_train, X1_2_test, y1_2_train, y1_2_test = train_test_split(X1_2, y1_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X1_2_train_scaled = scaler.fit_transform(X1_2_train)
X1_2_test_scaled = scaler.fit_transform(X1_2_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, no genre
X2_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y2_1 = df_cleaned_no_genre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X2_1_train, X2_1_test, y2_1_train, y2_1_test = train_test_split(X2_1, y2_1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X2_1_train_scaled = scaler.fit_transform(X2_1_train)
X2_1_test_scaled = scaler.fit_transform(X2_1_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, including genre
X2_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y2_2 = df_cleaned_genre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X2_2_train, X2_2_test, y2_2_train, y2_2_test = train_test_split(X2_2, y2_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X2_2_train_scaled = scaler.fit_transform(X2_2_train)
X2_2_test_scaled = scaler.fit_transform(X2_2_test)

Another 4 versions of the data the deep learning model

1. Max Peak Position, no genre (3__1 variables)
2. Max Peak Position, with genre (3_2 variables)
3. Max Rank Change, no genre (4_1 variables)
4. Max Rank Change, with genre (4_2 variables)

In [ ]:
# features and target for deep learning max_peak_position analysis, no genre
X3_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y3_1 = df_cleaned_no_genre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X3_1_train, X3_1_test, y3_1_train, y3_1_test = train_test_split(X3_1, y3_1, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X3_1_train_final, X3_1_val, y3_1_train_final, y3_1_val = train_test_split(X3_1_train, y3_1_train, test_size=0.2, random_state=42)

# normalizing 
scaler.fit(X3_1_train_final)
X3_1_train_scaled = scaler.transform(X3_1_train_final)
X3_1_val_scaled = scaler.transform(X3_1_val)
X3_1_test_scaled = scaler.transform(X3_1_test)

In [ ]:
# features and target for deep learning max_peak_position analysis, including genre
X3_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y3_2 = df_cleaned_genre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X3_2_train, X3_2_test, y3_2_train, y3_2_test = train_test_split(X3_2, y3_2, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X3_2_train_final, X3_2_val, y3_2_train_final, y3_2_val = train_test_split(X3_2_train, y3_2_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X3_2_train_final)
X3_2_train_scaled = scaler.transform(X3_2_train_final)
X3_2_val_scaled = scaler.transform(X3_2_val)
X3_2_test_scaled = scaler.transform(X3_2_test)

In [ ]:
# features and target for deep learning max_rank_change analysis, no genre
X4_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y4_1 = df_cleaned_no_genre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X4_1_train, X4_1_test, y4_1_train, y4_1_test = train_test_split(X4_1, y4_1, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X4_1_train_final, X4_1_val, y4_1_train_final, y4_1_val = train_test_split(X4_1_train, y4_1_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X4_1_train_final)
X4_1_train_scaled = scaler.transform(X4_1_train_final)
X4_1_val_scaled = scaler.transform(X4_1_val)
X4_1_test_scaled = scaler.transform(X4_1_test)

In [ ]:
# Prepare features and target for simple deep learning max_rank_change analysis, including genre
X4_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y4_2 = df_cleaned_genre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X4_2_train, X4_2_test, y4_2_train, y4_2_test = train_test_split(X4_2, y4_2, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X4_2_train_final, X4_2_val, y4_2_train_final, y4_2_val = train_test_split(X4_2_train, y4_2_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X4_2_train_final)
X4_2_train_scaled = scaler.transform(X4_2_train_final)
X4_2_val_scaled = scaler.transform(X4_2_val)
X4_2_test_scaled = scaler.transform(X4_2_test)

## Analysis

### XGBoost

**XGBoost | Max Peak Position - No Genre**

In [ ]:
# XGBoost for max_peak_position, no genre

xgb_model1_1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model1_1.fit(X1_1_train, y1_1_train)
y1_1_pred = xgb_model1_1.predict(X1_1_test)
y1_1_pred = np.clip(np.round(y1_1_pred), 1, 100)

rmse1_1 = np.sqrt(mean_squared_error(y1_1_test, y1_1_pred))
r2_1_1 = r2_score(y1_1_test, y1_1_pred)

print(f'RMSE: {rmse1_1:.3f}')
print(f'R²: {r2_1_1:.3f}')

In [ ]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb1_1 = GridSearchCV(estimator=xgb_model1_1,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_1.fit(X1_1_train, y1_1_train)

print("Best parameters:", grid_search_xgb1_1.best_params_)

In [ ]:
# hyperparameter tuning 2

param_grid2 = {
    'max_depth': [5, 6, 7],
    'learning_rate': [0.005, 0.01, 0.015,],
    'subsample': [0.75, 0.8, 0.85],
    'colsample_bytree': [1.0, 1.1, 1.2],
}

grid_search_xgb1_1 = GridSearchCV(estimator=xgb_model1_1,
                            param_grid=param_grid2,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_1.fit(X1_1_train, y1_1_train)

print("Best parameters:", grid_search_xgb1_1.best_params_)

In [ ]:
# hyperparameter tuning 3

param_grid3 = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.015],
    'subsample': [0.73, 0.74, 0.75],
    'colsample_bytree': [1.0],
}

grid_search_xgb1_1 = GridSearchCV(estimator=xgb_model1_1,
                            param_grid=param_grid3,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_1.fit(X1_1_train, y1_1_train)

print("Best parameters:", grid_search_xgb1_1.best_params_)

In [ ]:
# Extract best/final model for max_peak_position
best_xgb1_1 = grid_search_xgb1_1.best_estimator_

# predictions
y1_1_pred_best = best_xgb1_1.predict(X1_1_test)
y1_1_pred_best = np.clip(np.round(y1_1_pred_best), 1, 100)

# Evaluate XGBoost model
rmse1_1_best = np.sqrt(mean_squared_error(y1_1_test, y1_1_pred_best))
r2_1_1_best = r2_score(y1_1_test, y1_1_pred_best)

print(f'RMSE: {rmse1_1_best:.3f}')
print(f'R²: {r2_1_1_best:.3f}')

**XGBoost | Max Peak Position - With Genre**

In [ ]:
# XGBoost for max_peak_position, with genre

xgb_model1_2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model1_2.fit(X1_2_train, y1_2_train)
y1_2_pred = xgb_model1_2.predict(X1_2_test)
y1_2_pred = np.clip(np.round(y1_2_pred), 1, 100)

rmse1_2 = np.sqrt(mean_squared_error(y1_2_test, y1_2_pred))
r2_1_2 = r2_score(y1_2_test, y1_2_pred)

print(f'RMSE: {rmse1_2:.3f}')
print(f'R²: {r2_1_2:.3f}')

In [ ]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb1_2 = GridSearchCV(estimator=xgb_model1_2,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_2.fit(X1_2_train, y1_2_train)

print("Best parameters:", grid_search_xgb1_2.best_params_)

In [ ]:
# hyperparameter tuning 2

param_grid4 = {
    'max_depth': [8, 9, 10],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
}

grid_search_xgb1_2 = GridSearchCV(estimator=xgb_model1_2,
                            param_grid=param_grid4,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_2.fit(X1_2_train, y1_2_train)

print("Best parameters:", grid_search_xgb1_2.best_params_)

In [ ]:
# hyperparameter tuning 3

param_grid5 = {
    'max_depth': [7, 8],
    'learning_rate': [0.013, 0.015, 0.017],
    'subsample': [0.5, 0.6],
    'colsample_bytree': [0.8],
}

grid_search_xgb1_2 = GridSearchCV(estimator=xgb_model1_2,
                            param_grid=param_grid5,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_2.fit(X1_2_train, y1_2_train)

print("Best parameters:", grid_search_xgb1_2.best_params_)

In [ ]:
# Extract best/final model 
best_xgb1_2 = grid_search_xgb1_2.best_estimator_

# predictions
y1_2_pred_best = best_xgb1_2.predict(X1_2_test)
y1_2_pred_best = np.clip(np.round(y1_2_pred_best), 1, 100)

# Evaluate XGBoost model
rmse1_2_best = np.sqrt(mean_squared_error(y1_2_test, y1_2_pred_best))
r2_1_2_best = r2_score(y1_2_test, y1_2_pred_best)

print(f'RMSE: {rmse1_2_best:.3f}')
print(f'R²: {r2_1_2_best:.3f}')

**XGBoost | Max Rank Change - No Genre**

In [ ]:
# XGBoost for max_rank_change

xgb_model2_1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model2_1.fit(X2_1_train, y2_1_train)
y2_1_pred = xgb_model2_1.predict(X2_1_test)
y2_1_pred = np.clip(np.round(y2_1_pred), 1, 100)

rmse2_1 = np.sqrt(mean_squared_error(y2_1_test, y2_1_pred))
r2_2_1 = r2_score(y2_1_test, y2_1_pred)

print(f'RMSE: {rmse2_1:.3f}')
print(f'R²: {r2_2_1:.3f}')

In [ ]:
# hyperparameter tuning 1 for max_rank_change

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb2_1 = GridSearchCV(estimator=xgb_model2_1,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_1.fit(X2_1_train, y2_1_train)

print("Best parameters:", grid_search_xgb2_1.best_params_)

In [ ]:
# hyperparameter tuning 2 for max_rank_change

param_grid4 = {
    'max_depth': [3],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [1.0, 1.1, 1.2],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

grid_search_xgb2_1 = GridSearchCV(estimator=xgb_model2_1,
                            param_grid=param_grid4,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_1.fit(X2_1_train, y2_1_train)

print("Best parameters:", grid_search_xgb2_1.best_params_)

In [ ]:
# hyperparameter tuning 3 for max_rank_change

param_grid5 = {
    'max_depth': [3],
    'learning_rate': [0.003, 0.005, 0.007],
    'subsample': [1.0],
    'colsample_bytree': [0.9],
        }

grid_search_xgb2_1 = GridSearchCV(estimator=xgb_model2_1,
                            param_grid=param_grid5,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_1.fit(X2_1_train, y2_1_train)

print("Best parameters:", grid_search_xgb2_1.best_params_)

In [ ]:
# Extract best/final model  for max_rank_change
best_xgb2_1 = grid_search_xgb2_1.best_estimator_

y2_1_pred_best = best_xgb2_1.predict(X2_1_test)
y2_1_pred_best = np.clip(np.round(y2_1_pred_best), 1, 100)

# Evaluate XGBoost model
rmse2_1_best = np.sqrt(mean_squared_error(y2_1_test, y2_1_pred_best))
r2_2_1_best = r2_score(y2_1_test, y2_1_pred_best)

print(f'RMSE: {rmse2_1_best:.3f}')
print(f'R²: {r2_2_1_best:.3f}')

**XGBoost | Max Rank Change - With Genre**

In [ ]:
# XGBoost for max_rank_change

xgb_model2_2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model2_2.fit(X2_2_train, y2_2_train)
y2_2_pred = xgb_model2_2.predict(X2_2_test)
y2_2_pred = np.clip(np.round(y2_2_pred), 1, 100)

rmse2_2 = np.sqrt(mean_squared_error(y2_2_test, y2_2_pred))
r2_2_2 = r2_score(y2_2_test, y2_2_pred)

print(f'RMSE: {rmse2_2:.3f}')
print(f'R²: {r2_2_2:.3f}')

In [ ]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb2_2 = GridSearchCV(estimator=xgb_model2_2,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_2.fit(X2_2_train, y2_2_train)

print("Best parameters:", grid_search_xgb2_2.best_params_)

In [ ]:
# hyperparameter tuning 2

param_grid6 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [1.0, 1.1, 1.2],
    'colsample_bytree': [0.6, 0.7, 0.8],
}

grid_search_xgb2_2 = GridSearchCV(estimator=xgb_model2_2,
                            param_grid=param_grid6,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_2.fit(X2_2_train, y2_2_train)

print("Best parameters:", grid_search_xgb2_2.best_params_)

In [ ]:
# hyperparameter tuning 3

param_grid7 = {
    'max_depth': [2],
    'learning_rate': [0.003, 0.005, 0.007],
    'subsample': [1.0],
    'colsample_bytree': [0.8],
}

grid_search_xgb2_2 = GridSearchCV(estimator=xgb_model2_2,
                            param_grid=param_grid7,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_2.fit(X2_2_train, y2_2_train)

print("Best parameters:", grid_search_xgb2_2.best_params_)

In [ ]:
# Extract best/final model  for max_rank_change
best_xgb2_2 = grid_search_xgb2_2.best_estimator_

y2_2_pred_best = best_xgb2_2.predict(X2_2_test)
y2_2_pred_best = np.clip(np.round(y2_2_pred_best), 1, 100)

# Evaluate XGBoost model
rmse2_2_best = np.sqrt(mean_squared_error(y2_2_test, y2_2_pred_best))
r2_2_2_best = r2_score(y2_2_test, y2_2_pred_best)

print(f'RMSE: {rmse2_2_best:.3f}')
print(f'R²: {r2_2_2_best:.3f}')

#### XGBoost Summary

Using XGBoost, including the genre features slightly improved model performance. However, in all cases the r^2 values for the model were less than 0.02, indicating a near-zero fit of the model to the test data. XGBoost will not be explored further.

### k-Nearest Neighbors

**k-Nearest Neighbors | Max Peak Position - No Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search1_1 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search1_1.fit(X1_1_train_scaled, y1_1_train)

# Get best parameters and score
standard_best_params1_1 = grid_search1_1.best_params_
standard_best_score1_1 = grid_search1_1.best_score_
print(f"Best parameters (standard metrics): {standard_best_params1_1}")
print(f"Best cross-validation accuracy: {standard_best_score1_1:.4f}")


In [ ]:
# final model with best parameters
final_model1_1 = grid_search1_1.best_estimator_


# Make predictions on test set
y1_1_pred = final_model1_1.predict(X1_1_test_scaled)


# Evaluate the model
accuracy1_1 = accuracy_score(y1_1_test, y1_1_pred)

print(f"Best accuracy: {accuracy1_1:.4f}")

**k-Nearest Neighbors | Max Peak Position - With Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search1_2 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search1_2.fit(X1_2_train_scaled, y1_2_train)

# Get best parameters and score
standard_best_params1_2 = grid_search1_2.best_params_
standard_best_score1_2 = grid_search1_2.best_score_
print(f"Best parameters (standard metrics): {standard_best_params1_2}")
print(f"Best cross-validation accuracy: {standard_best_score1_2:.4f}")


In [ ]:
# final model with best parameters
final_model1_2 = grid_search1_2.best_estimator_


# Make predictions on test set
y1_2_pred = final_model1_2.predict(X1_2_test_scaled)


# Evaluate the model
accuracy1_2 = accuracy_score(y1_2_test, y1_2_pred)

print(f"Best accuracy: {accuracy1_2:.4f}")

**k-Nearest Neighbors | Max Rank Change - No Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search2_1 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search2_1.fit(X2_1_train_scaled, y2_1_train)

# Get best parameters and score
standard_best_params2_1 = grid_search2_1.best_params_
standard_best_score2_1 = grid_search2_1.best_score_
print(f"Best parameters (standard metrics): {standard_best_params2_1}")
print(f"Best cross-validation accuracy: {standard_best_score2_1:.4f}")


In [ ]:
# final model with best parameters
final_model2_1 = grid_search2_1.best_estimator_


# Make predictions on test set
y1_1_pred = final_model1_1.predict(X2_1_test_scaled)


# Evaluate the model
accuracy2_1 = accuracy_score(y2_1_test, y2_1_pred)

print(f"Best accuracy: {accuracy2_1:.4f}")

**k-Nearest Neighbors | Max Rank Change - With Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search2_2 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search2_2.fit(X2_2_train_scaled, y2_2_train)

# Get best parameters and score
standard_best_params2_2 = grid_search2_2.best_params_
standard_best_score2_2 = grid_search2_2.best_score_
print(f"Best parameters (standard metrics): {standard_best_params2_2}")
print(f"Best cross-validation accuracy: {standard_best_score2_2:.4f}")


In [ ]:
# final model with best parameters
final_model2_2 = grid_search2_2.best_estimator_


# Make predictions on test set
y2_2_pred = final_model2_2.predict(X2_2_test_scaled)


# Evaluate the model
accuracy2_2 = accuracy_score(y2_2_test, y2_2_pred)

print(f"Best accuracy: {accuracy2_2:.4f}")

#### k-NN Summary

The k-NN models performed better when the genre features were excluded. However, none of the models achieved an accuracy above 7% for either peak position or maximum rank change so k-NN will not be explored further for this project.

### Deep Learning

**Deep Learning | Max Peak Position - No Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X3_1_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline3_1 = baseline_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model3_1 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X3_1_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model3_1 = bnorm_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, NO GENRE")

print("\n=== Baseline Model ===")
train_scores3_1 = baseline_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores3_1   = baseline_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores3_1[1]:.4f}, Train MSE: {train_scores3_1[2]:.4f}")
print(f"Val   MAE: {val_scores3_1[1]:.4f}, Val   MSE: {val_scores3_1[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn3_1 = bnorm_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_bn3_1   = bnorm_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_bn3_1[1]:.4f}, Train MSE: {train_scores_bn3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn3_1[1]:.4f}, Val   MSE: {val_scores_bn3_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


**Deep Learning | Max Peak Position - With Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model3_2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X3_2_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline3_2 = baseline_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model3_2 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X3_2_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model3_2 = bnorm_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_2 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_2_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_2 = reg_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, WITH GENRE")

print("\n=== Baseline Model ===")
train_scores3_2 = baseline_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores3_2   = baseline_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores3_2[1]:.4f}, Train MSE: {train_scores3_2[2]:.4f}")
print(f"Val   MAE: {val_scores3_2[1]:.4f}, Val   MSE: {val_scores3_2[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn3_2 = bnorm_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores_bn3_2   = bnorm_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores_bn3_2[1]:.4f}, Train MSE: {train_scores_bn3_2[2]:.4f}")
print(f"Val   MAE: {val_scores_bn3_2[1]:.4f}, Val   MSE: {val_scores_bn3_2[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg3_2 = reg_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores_reg3_2   = reg_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_2[1]:.4f}, Train MSE: {train_scores_reg3_2[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_2[1]:.4f}, Val   MSE: {val_scores_reg3_2[2]:.4f}")


**Deep Learning | Max Rank Change - No Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X4_1_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline4_1 = baseline_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model4_1 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X4_1_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model4_1 = bnorm_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_1 = reg_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, NO GENRE")

print("\n=== Baseline Model ===")
train_scores4_1 = baseline_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores4_1   = baseline_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores4_1[1]:.4f}, Train MSE: {train_scores4_1[2]:.4f}")
print(f"Val   MAE: {val_scores4_1[1]:.4f}, Val   MSE: {val_scores4_1[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn4_1 = bnorm_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_bn4_1   = bnorm_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_bn4_1[1]:.4f}, Train MSE: {train_scores_bn4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn4_1[1]:.4f}, Val   MSE: {val_scores_bn4_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")


**Deep Learning | Max Rank Change - With Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model4_2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X4_2_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline4_2 = baseline_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model4_2 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X4_2_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model4_2 = bnorm_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_2 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_2_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_2 = reg_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, NO GENRE")

print("\n=== Baseline Model ===")
train_scores4_2 = baseline_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores4_2   = baseline_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores4_2[1]:.4f}, Train MSE: {train_scores4_2[2]:.4f}")
print(f"Val   MAE: {val_scores4_2[1]:.4f}, Val   MSE: {val_scores4_2[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn4_2 = bnorm_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores_bn4_2   = bnorm_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores_bn4_1[1]:.4f}, Train MSE: {train_scores_bn4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn4_1[1]:.4f}, Val   MSE: {val_scores_bn4_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg4_2 = reg_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores_reg4_2   = reg_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_2[1]:.4f}, Train MSE: {train_scores_reg4_2[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_2[1]:.4f}, Val   MSE: {val_scores_reg4_2[2]:.4f}")


**Deep Learning | Optimizing Regularized Models**

Max Peak Position

In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("    Adding an additional deep layer")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


Adding a layer resulted in a larger MAE for both training and validation. Removing the additional layer and adding kernels.

In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("Reverting to 2 deep layers, increasing to 128 kernels per layer")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


Increasing kernels per layer gave a result very similar to the original model. Leaving 128 kernels per layer and increasing the dropout rate.

In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.6

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("2 deep layers,  kernels per layer, increased dropout from 0.4 to 0.6")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.6

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("3 deep layers, 128 kernels per layer, 0.6 dropout rate")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


Max Rank Change

In [ ]:
# adding an additional layer 

l2_reg = 1e-4
dropout_rate = 0.6

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:

print("=== Regularized Model (L2 + Dropout) ===")
print("    Adding an additional deep layer")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")


## Evaluation

### Summary of Model Performance

### Business Insight/Recommendation 1

### Business Insight/Recommendation 2

### Business Insight/Recommendation 3

### Tableau Dashboard link

## Conclusion and Next Steps
Text here